# **Feast**

In [32]:
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
from feast import FeatureStore, Entity, FeatureView, ValueType, FileSource
from feast.types import Float64, Int64
from feast import FeatureStore
from joblib import dump
from feast.infra.offline_stores.file_source import SavedDatasetFileStorage
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.ensemble import StackingRegressor,RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from warnings import filterwarnings
filterwarnings('ignore')


In [2]:
data = pd.read_csv("C:/Users/chhon/Python Notebooks/BCS_Win_Factor_Summer_Project/df_features.csv")

In [3]:
data

,player_id,match_id,start_date,order_seen,unique_name,fantasy_score_total,match_type_MDM,match_type_ODI,match_type_ODM,match_type_T20,...,all_rounder_10,impact_bat_1,impact_bat_5,impact_bat_10,impact_bowl_1,impact_bowl_5,impact_bowl_10,fielding_impact_1,fielding_impact_5,fielding_impact_10
0,3444,63963,2001-12-19,5,MR Ramprakash,71,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
1,80,63963,2001-12-19,0,R Dravid,21,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
2,5246,63963,2001-12-19,0,SC Ganguly,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
3,9622,63963,2001-12-19,11,RKJ Dawson,4,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
4,10185,63963,2001-12-19,0,VVS Laxman,26,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416704,9505,1473500,2025-05-20,3,Urvil Patel,2,0,0,0,1,...,18.792789,27.000000,19.747512,18.836293,0.00000,0.0,0.0,0.0,3.551681,5.590619
416705,3563,1485778,2025-05-20,0,R Poddar,2,0,0,0,1,...,0.000000,15.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,5.614559,0.000000
416706,8320,1485778,2025-05-20,4,N Chantam,23,0,0,0,1,...,14.650822,22.571429,0.000000,0.000000,0.00000,0.0,0.0,20.0,11.307957,7.854720
416707,1839,1462374,2025-05-20,0,GA Elwiss,116,0,0,1,0,...,40.586116,12.173913,10.076169,9.163169,4.77907,0.0,0.0,5.0,2.645377,1.705186


In [4]:
data

,player_id,match_id,start_date,order_seen,unique_name,fantasy_score_total,match_type_MDM,match_type_ODI,match_type_ODM,match_type_T20,...,all_rounder_10,impact_bat_1,impact_bat_5,impact_bat_10,impact_bowl_1,impact_bowl_5,impact_bowl_10,fielding_impact_1,fielding_impact_5,fielding_impact_10
0,3444,63963,2001-12-19,5,MR Ramprakash,71,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
1,80,63963,2001-12-19,0,R Dravid,21,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
2,5246,63963,2001-12-19,0,SC Ganguly,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
3,9622,63963,2001-12-19,11,RKJ Dawson,4,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
4,10185,63963,2001-12-19,0,VVS Laxman,26,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416704,9505,1473500,2025-05-20,3,Urvil Patel,2,0,0,0,1,...,18.792789,27.000000,19.747512,18.836293,0.00000,0.0,0.0,0.0,3.551681,5.590619
416705,3563,1485778,2025-05-20,0,R Poddar,2,0,0,0,1,...,0.000000,15.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,5.614559,0.000000
416706,8320,1485778,2025-05-20,4,N Chantam,23,0,0,0,1,...,14.650822,22.571429,0.000000,0.000000,0.00000,0.0,0.0,20.0,11.307957,7.854720
416707,1839,1462374,2025-05-20,0,GA Elwiss,116,0,0,1,0,...,40.586116,12.173913,10.076169,9.163169,4.77907,0.0,0.0,5.0,2.645377,1.705186


In [5]:
predictors_df = data.drop(columns=['fantasy_score_total','match_id','start_date','unique_name'], axis=1)
target_df = data['fantasy_score_total']

In [6]:
predictors_df

,player_id,order_seen,match_type_MDM,match_type_ODI,match_type_ODM,match_type_T20,match_type_Test,impact_bat,impact_bowl,runs_scored_1,...,all_rounder_10,impact_bat_1,impact_bat_5,impact_bat_10,impact_bowl_1,impact_bowl_5,impact_bowl_10,fielding_impact_1,fielding_impact_5,fielding_impact_10
0,3444,5,0,0,0,0,1,4.436090,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
1,80,0,0,0,0,0,1,0.491803,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
2,5246,0,0,0,0,0,1,0.000000,3.804124,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
3,9622,11,0,0,0,0,1,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
4,10185,0,0,0,0,0,1,6.888889,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416704,9505,3,0,0,0,1,0,0.000000,0.000000,31.0,...,18.792789,27.000000,19.747512,18.836293,0.00000,0.0,0.0,0.0,3.551681,5.590619
416705,3563,0,0,0,0,1,0,4.000000,0.000000,6.0,...,0.000000,15.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,5.614559,0.000000
416706,8320,4,0,0,0,1,0,15.400000,0.000000,46.0,...,14.650822,22.571429,0.000000,0.000000,0.00000,0.0,0.0,20.0,11.307957,7.854720
416707,1839,0,0,0,1,0,0,10.788732,3.625000,82.0,...,40.586116,12.173913,10.076169,9.163169,4.77907,0.0,0.0,5.0,2.645377,1.705186


In [7]:
target_df

0          71
1          21
2           0
3           4
4          26
         ... 
416704      2
416705      2
416706     23
416707    116
416708     14
Name: fantasy_score_total, Length: 416709, dtype: int64

In [8]:
timestamps = pd.date_range(
    end=pd.Timestamp.now(), 
    periods=len(data), 
    freq='h').to_frame(name="event_timestamp", index=False)

In [9]:
datalen = len(data)
ids = list(range(datalen))

In [10]:
entity_id = pd.DataFrame(data = ids, columns=['entity_id'])

In [11]:
predictors_df = pd.concat([entity_id, predictors_df], axis=1)
target_df = pd.concat([entity_id, target_df], axis=1)

In [12]:
predictors_df = pd.concat([timestamps, predictors_df], axis=1)
target_df = pd.concat([timestamps, target_df], axis=1)

In [13]:
predictors_df

,event_timestamp,entity_id,player_id,order_seen,match_type_MDM,match_type_ODI,match_type_ODM,match_type_T20,match_type_Test,impact_bat,...,all_rounder_10,impact_bat_1,impact_bat_5,impact_bat_10,impact_bowl_1,impact_bowl_5,impact_bowl_10,fielding_impact_1,fielding_impact_5,fielding_impact_10
0,1977-12-20 05:55:26.500216,0,3444,5,0,0,0,0,1,4.436090,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
1,1977-12-20 06:55:26.500216,1,80,0,0,0,0,0,1,0.491803,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
2,1977-12-20 07:55:26.500216,2,5246,0,0,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
3,1977-12-20 08:55:26.500216,3,9622,11,0,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
4,1977-12-20 09:55:26.500216,4,10185,0,0,0,0,0,1,6.888889,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416704,2025-07-03 21:55:26.500216,416704,9505,3,0,0,0,1,0,0.000000,...,18.792789,27.000000,19.747512,18.836293,0.00000,0.0,0.0,0.0,3.551681,5.590619
416705,2025-07-03 22:55:26.500216,416705,3563,0,0,0,0,1,0,4.000000,...,0.000000,15.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,5.614559,0.000000
416706,2025-07-03 23:55:26.500216,416706,8320,4,0,0,0,1,0,15.400000,...,14.650822,22.571429,0.000000,0.000000,0.00000,0.0,0.0,20.0,11.307957,7.854720
416707,2025-07-04 00:55:26.500216,416707,1839,0,0,0,1,0,0,10.788732,...,40.586116,12.173913,10.076169,9.163169,4.77907,0.0,0.0,5.0,2.645377,1.705186


In [14]:
target_df

,event_timestamp,entity_id,fantasy_score_total
0,1977-12-20 05:55:26.500216,0,71
1,1977-12-20 06:55:26.500216,1,21
2,1977-12-20 07:55:26.500216,2,0
3,1977-12-20 08:55:26.500216,3,4
4,1977-12-20 09:55:26.500216,4,26
...,...,...,...
416704,2025-07-03 21:55:26.500216,416704,2
416705,2025-07-03 22:55:26.500216,416705,2
416706,2025-07-03 23:55:26.500216,416706,23
416707,2025-07-04 00:55:26.500216,416707,116


In [15]:
predictors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416709 entries, 0 to 416708
Data columns (total 62 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   event_timestamp     416709 non-null  datetime64[ns]
 1   entity_id           416709 non-null  int64         
 2   player_id           416709 non-null  int64         
 3   order_seen          416709 non-null  int64         
 4   match_type_MDM      416709 non-null  int64         
 5   match_type_ODI      416709 non-null  int64         
 6   match_type_ODM      416709 non-null  int64         
 7   match_type_T20      416709 non-null  int64         
 8   match_type_Test     416709 non-null  int64         
 9   impact_bat          416709 non-null  float64       
 10  impact_bowl         416709 non-null  float64       
 11  runs_scored_1       416709 non-null  float64       
 12  runs_scored_5       416709 non-null  float64       
 13  runs_scored_10      416709 no

In [16]:
predictors_df.columns

Index(['event_timestamp', 'entity_id', 'player_id', 'order_seen',
       'match_type_MDM', 'match_type_ODI', 'match_type_ODM', 'match_type_T20',
       'match_type_Test', 'impact_bat', 'impact_bowl', 'runs_scored_1',
       'runs_scored_5', 'runs_scored_10', 'fours_scored_1', 'fours_scored_5',
       'fours_scored_10', 'sixes_scored_1', 'sixes_scored_5',
       'sixes_scored_10', 'catches_taken_1', 'catches_taken_5',
       'catches_taken_10', 'run_out_direct_1', 'run_out_direct_5',
       'run_out_direct_10', 'run_out_throw_1', 'run_out_throw_5',
       'run_out_throw_10', 'stumpings_done_1', 'stumpings_done_5',
       'stumpings_done_10', 'wickets_taken_1', 'wickets_taken_5',
       'wickets_taken_10', 'bowled_done_1', 'bowled_done_5', 'bowled_done_10',
       'lbw_done_1', 'lbw_done_5', 'lbw_done_10', 'maidens_1', 'maidens_5',
       'maidens_10', 'strike_rate_1', 'strike_rate_5', 'strike_rate_10',
       'economy_rate_1', 'economy_rate_5', 'economy_rate_10', 'all_rounder_1',
      

In [17]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416709 entries, 0 to 416708
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   event_timestamp      416709 non-null  datetime64[ns]
 1   entity_id            416709 non-null  int64         
 2   fantasy_score_total  416709 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 9.5 MB


In [18]:
# !feast init feast_repo

In [19]:
pwd

'c:\\Users\\chhon\\Python Notebooks\\BCS_Win_Factor_Summer_Project'

In [20]:
cd feast_repo

c:\Users\chhon\Python Notebooks\BCS_Win_Factor_Summer_Project\feast_repo


In [21]:
cd feature_repo

c:\Users\chhon\Python Notebooks\BCS_Win_Factor_Summer_Project\feast_repo\feature_repo


In [22]:
pwd

'c:\\Users\\chhon\\Python Notebooks\\BCS_Win_Factor_Summer_Project\\feast_repo\\feature_repo'

In [24]:
!feast entities list

NAME    DESCRIPTION    TYPE
player                 ValueType.UNKNOWN


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [25]:
!feast feature-views list

NAME                        ENTITIES    TYPE
predictors_df_feature_view  {'player'}  FeatureView
target_df_feature_view      {'player'}  FeatureView


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [26]:
store = FeatureStore(repo_path="C:/Users/chhon/Python Notebooks/BCS_Win_Factor_Summer_Project/feast_repo/feature_repo/")

In [27]:
entity_df = pd.read_parquet(path="C:/Users/chhon/Python Notebooks/BCS_Win_Factor_Summer_Project/feast_repo/feature_repo/data/target.parquet")  

In [28]:
training_data = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "predictors_df_feature_view:player_id",
        "predictors_df_feature_view:order_seen",
        "predictors_df_feature_view:match_type_MDM",
        "predictors_df_feature_view:match_type_ODI",
        "predictors_df_feature_view:match_type_T20",
        "predictors_df_feature_view:match_type_Test",
        "predictors_df_feature_view:impact_bat",
        "predictors_df_feature_view:impact_bowl",
        "predictors_df_feature_view:runs_scored_1",
        "predictors_df_feature_view:runs_scored_5",
        "predictors_df_feature_view:runs_scored_10",
        "predictors_df_feature_view:fours_scored_1",
        "predictors_df_feature_view:fours_scored_5",
        "predictors_df_feature_view:fours_scored_10",
        "predictors_df_feature_view:sixes_scored_1",
        "predictors_df_feature_view:sixes_scored_5",
        "predictors_df_feature_view:sixes_scored_10",
        "predictors_df_feature_view:catches_taken_1",
        "predictors_df_feature_view:catches_taken_5",
        "predictors_df_feature_view:catches_taken_10",
        "predictors_df_feature_view:run_out_direct_1",
        "predictors_df_feature_view:run_out_direct_5",
        "predictors_df_feature_view:run_out_direct_10",
        "predictors_df_feature_view:run_out_throw_1",
        "predictors_df_feature_view:run_out_throw_5",
        "predictors_df_feature_view:run_out_throw_10",
        "predictors_df_feature_view:stumpings_done_1",
        "predictors_df_feature_view:stumpings_done_5",
        "predictors_df_feature_view:stumpings_done_10",
        "predictors_df_feature_view:wickets_taken_1",
        "predictors_df_feature_view:wickets_taken_5",
        "predictors_df_feature_view:wickets_taken_10",
        "predictors_df_feature_view:bowled_done_1",
        "predictors_df_feature_view:bowled_done_5",
        "predictors_df_feature_view:bowled_done_10",
        "predictors_df_feature_view:lbw_done_1",
        "predictors_df_feature_view:lbw_done_5",
        "predictors_df_feature_view:lbw_done_10",
        "predictors_df_feature_view:maidens_1",
        "predictors_df_feature_view:maidens_5",
        "predictors_df_feature_view:maidens_10",
        "predictors_df_feature_view:strike_rate_1",
        "predictors_df_feature_view:strike_rate_5",
        "predictors_df_feature_view:strike_rate_10",
        "predictors_df_feature_view:economy_rate_1",
        "predictors_df_feature_view:economy_rate_5",
        "predictors_df_feature_view:economy_rate_10",
        "predictors_df_feature_view:all_rounder_1",
        "predictors_df_feature_view:all_rounder_5",
        "predictors_df_feature_view:all_rounder_10",
        "predictors_df_feature_view:impact_bat_1",
        "predictors_df_feature_view:impact_bat_5",
        "predictors_df_feature_view:impact_bat_10",
        "predictors_df_feature_view:impact_bowl_1",
        "predictors_df_feature_view:impact_bowl_5",
        "predictors_df_feature_view:impact_bowl_10",
        "predictors_df_feature_view:fielding_impact_1",
        "predictors_df_feature_view:fielding_impact_5",
        "predictors_df_feature_view:fielding_impact_10"
    ]
)

In [ ]:
dataset = store.create_saved_dataset(
    from_=training_data,
    name="df_feature",
    storage=SavedDatasetFileStorage("C:/Users/chhon/Python Notebooks/BCS_Win_Factor_Summer_Project/feast_repo/feature_repo/data/df_feature.parquet")
)

c:\Users\chhon\Python Notebooks\BCS_Win_Factor_Summer_Project\venv\Lib\site-packages\feast\feature_store.py:1223: RuntimeWarning: Saving dataset is an experimental feature. This API is unstable and it could and most probably will be changed in the future. We do not guarantee that future changes will maintain backward compatibility.
  warnings.warn(


In [29]:
training_data.to_df()

,event_timestamp,entity_id,fantasy_score_total,player_id,order_seen,match_type_MDM,match_type_ODI,match_type_T20,match_type_Test,impact_bat,...,all_rounder_10,impact_bat_1,impact_bat_5,impact_bat_10,impact_bowl_1,impact_bowl_5,impact_bowl_10,fielding_impact_1,fielding_impact_5,fielding_impact_10
0,1977-12-20 04:23:21.371088+00:00,0,71,3444,5,0,0,0,1,4.436090,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
1,1977-12-20 05:23:21.371088+00:00,1,21,80,0,0,0,0,1,0.491803,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
2,1977-12-20 06:23:21.371088+00:00,2,0,5246,0,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
3,1977-12-20 07:23:21.371088+00:00,3,4,9622,11,0,0,0,1,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
4,1977-12-20 08:23:21.371088+00:00,4,26,10185,0,0,0,0,1,6.888889,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416704,2025-07-03 20:23:21.371088+00:00,416704,2,9505,3,0,0,1,0,0.000000,...,18.792789,27.000000,19.747512,18.836293,0.00000,0.0,0.0,0.0,3.551681,5.590619
416705,2025-07-03 21:23:21.371088+00:00,416705,2,3563,0,0,0,1,0,4.000000,...,0.000000,15.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0,5.614559,0.000000
416706,2025-07-03 22:23:21.371088+00:00,416706,23,8320,4,0,0,1,0,15.400000,...,14.650822,22.571429,0.000000,0.000000,0.00000,0.0,0.0,20.0,11.307957,7.854720
416707,2025-07-03 23:23:21.371088+00:00,416707,116,1839,0,0,0,0,0,10.788732,...,40.586116,12.173913,10.076169,9.163169,4.77907,0.0,0.0,5.0,2.645377,1.705186


In [30]:
store = FeatureStore(repo_path="C:/Users/chhon/Python Notebooks/BCS_Win_Factor_Summer_Project/feast_repo/feature_repo")

In [31]:
training_df = store.get_saved_dataset(name="df_feature").to_df()
label = training_df['fantasy_score_total']
features = training_df.drop(columns=['fantasy_score_total', 'event_timestamp', 'entity_id'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)
model = XGBRegressor(
    learning_rate=0.01,
    max_depth=8,
    n_estimators=650,
    subsample=0.75,
    colsample_bytree=0.7,
    reg_alpha=6,
    reg_lambda= 8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

model.fit(X_train, y_train)

dump(value=model, filename="model.joblib")

['model.joblib']

In [33]:
store = FeatureStore(repo_path="C:/Users/chhon/Python Notebooks/BCS_Win_Factor_Summer_Project/feast_repo/feature_repo")

store.materialize_incremental(end_date=datetime.now())

Materializing 2 feature views to 2025-07-04 02:14:15+00:00 into the sqlite online store.

predictors_df_feature_view from 2025-06-26 20:44:15+00:00 to 2025-07-04 02:14:15+00:00:
target_df_feature_view from 2025-06-26 20:44:22+00:00 to 2025-07-04 02:14:15+00:00:
